[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/CRM-jupyter/blob/main/CRM_jupyter.ipynb)

In [ ]:
%cd /content
!git clone -b 22.04-dev https://github.com/NVIDIA/apex
%cd /content/apex
!pip install -v -e .

%cd /content
!git clone -b dev https://github.com/camenduru/Open-Sora
%cd /content/Open-Sora
!pip install -v -e .

!pip install packaging ninja colossalai mmengine
!pip install flash-attn --no-build-isolation
!pip install https://download.pytorch.org/whl/cu121/xformers-0.0.25-cp310-cp310-manylinux2014_x86_64.whl

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hpcai-tech/Open-Sora/resolve/main/OpenSora-v1-16x256x256.pth -d /content/models -o OpenSora-v1-16x256x256.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hpcai-tech/Open-Sora/resolve/main/OpenSora-v1-HQ-16x256x256.pth -d /content/models -o OpenSora-v1-HQ-16x256x256.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hpcai-tech/Open-Sora/resolve/main/OpenSora-v1-HQ-16x512x512.pth -d /content/models -o OpenSora-v1-HQ-16x512x512.pth

In [ ]:
%cd /content/Open-Sora

# Sample 16x256x256 (5s/sample)
!torchrun --standalone --nproc_per_node 1 scripts/inference.py configs/opensora/inference/16x256x256.py --ckpt-path /content/models/OpenSora-v1-16x256x256.pth

In [ ]:
# Sample 16x512x512 (20s/sample, 100 time steps)
!torchrun --standalone --nproc_per_node 1 scripts/inference.py configs/opensora/inference/16x512x512.py --ckpt-path /content/models/OpenSora-v1-HQ-16x512x512.pth

# Sample 64x512x512 (40s/sample, 100 time steps)
!torchrun --standalone --nproc_per_node 1 scripts/inference.py configs/opensora/inference/64x512x512.py --ckpt-path /content/models/OpenSora-v1-HQ-16x512x512.pth

# Sample 64x512x512 with sequence parallelism (30s/sample, 100 time steps)
# sequence parallelism is enabled automatically when nproc_per_node is larger than 1
!torchrun --standalone --nproc_per_node 2 scripts/inference.py configs/opensora/inference/64x512x512.py --ckpt-path /content/models/OpenSora-v1-HQ-16x512x512.pth